In [77]:
from sdv.demo import load_timeseries_demo
import pandas as pd

In [78]:
data = pd.read_csv('./AAPL Historical Data.csv')

In [103]:
data.head()

,Date,Price
0,11/01/2013,19.86
1,12/01/2013,20.04
2,01/01/2014,17.88
3,02/01/2014,18.79
4,03/01/2014,19.17


In [104]:
from sdv.timeseries import PAR


In [105]:
model = PAR(
  )

In [106]:
model.fit(data)

In [109]:
data_new = model.sample()

,Date,Price
0,03/01/2014,57.496768
1,07/01/2014,37.041698
2,12/01/2013,27.912242
3,08/01/2014,24.811002
4,06/01/2020,22.741953
...,...,...
94,08/01/2014,39.764851
95,11/01/2019,32.965641
96,01/01/2017,42.436336
97,11/01/2018,41.376652
